# Exploring the DGGRID tool via Python

[![Population Gridded](day-04-hexa.png)](https://twitter.com/allixender/status/1324055326111485959)

## DGGRIDv7

DGGRID is available under the GNU AFFERO GENERAL PUBLIC LICENSE

[DGGRID](https://www.discreteglobalgrids.org/software/) is a free software program for creating and manipulating Discrete Global Grids created and maintained by Kevin Sahr. DGGRID version 7.0 was released in September, 2019.

- [DGGRID Version 7.0 on GitHub](https://github.com/sahrk/DGGRID)
- [DGGRID User Manual](https://webpages.sou.edu/~sahrk/docs/dggridManualV70.pdf)


## dggrid4py - a Python library to run highlevel functions of DGGRID

https://github.com/allixender/dggrid4py

You need the ddgrid tool compiled available on the system.

Besides some lowlevel access influence the dggrid operations' metafile creation, a few highlevel functions are integrated to work with the more comfortable geopython libraries, like shapely and geopandas

- grid_cell_polygons_for_extent(): fill extent/subset with cells at resolution (clip or world)
- grid_cell_polygons_from_cellids(): geometry_from_cellid for dggs at resolution (from id list)
- grid_cellids_for_extent(): get_all_indexes/cell_ids for dggs at resolution (clip or world)
- cells_for_geo_points(): poly_outline for point/centre at resolution

In [1]:
import os
from julia.api import Julia
from dggrid4py import DGGRIDv7

jl = Julia(compiled_modules=False)

jl.eval("using DGGRID7_jll")
dirs = jl.eval("DGGRID7_jll.LIBPATH_list")

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [2]:
# for Windows path separator is ";" and the variable is PATH
# for linux the path separator is ":" and the variable is LD_LIBRARY_PATH
path_update = ":".join(dirs)
os.environ["LD_LIBRARY_PATH"] = path_update

dggrid_exec = jl.eval("DGGRID7_jll.get_dggrid_path()")

NameError: name 'dirs' is not defined

In [3]:
dggrid = DGGRIDv7(executable=dggrid_exec, working_dir=os.curdir, capture_logs=False, silent=False)

NameError: name 'dggrid_exec' is not defined

In [4]:
import geopandas
import shapely

from dggrid4py import DGGRIDv7

In [5]:
gdf1 = dggrid.grid_cell_polygons_for_extent('ISEA4T', 3)

NameError: name 'dggrid' is not defined

In [6]:
gdf1 

NameError: name 'gdf1' is not defined

In [ ]:
gdf1.plot()

In [ ]:
import geoviews as gv
import geoviews.feature as gf
from geoviews import opts, tile_sources as gvts
import cartopy.crs as ccrs

import matplotlib.pyplot as plt
%matplotlib inline

gv.extension('matplotlib')

gv.output(fig='png', size=300)

In [ ]:
(gf.ocean * gf.coastline * gf.land).opts( projection=ccrs.Orthographic(27, 48) )

In [ ]:
gf.ocean * gf.coastline * gf.land * gv.Polygons(
    gdf1, vdims=[]
).opts( projection=ccrs.Orthographic(27, 48) )

In [ ]:
from typing import Union, List
from shapely.geometry import mapping, Polygon, GeometryCollection
from shapely import affinity

# https://towardsdatascience.com/around-the-world-in-80-lines-crossing-the-antimeridian-with-python-and-shapely-c87c9b6e1513
# https://gist.github.com/PawaritL/ec7136c0b718ca65db6df1c33fd1bb11

def check_crossing(lon1: float, lon2: float, validate: bool = True):
    """
    Assuming a minimum travel distance between two provided longitude coordinates,
    checks if the 180th meridian (antimeridian) is crossed.
    """
    if validate and any(abs(x) > 180.0 for x in [lon1, lon2]):
        raise ValueError("longitudes must be in degrees [-180.0, 180.0]")   
    return abs(lon2 - lon1) > 180.0

def check_for_geom(geom):
    crossed = False
    p_init = geom.exterior.coords[0]

    for p in range(1, len(geom.exterior.coords)):
        px = geom.exterior.coords[p]
        # print(px)

        if check_crossing(p_init[0], px[0]):
            crossed = True
        p_init = px
    
    return crossed


In [ ]:
gdf1['crossed'] = gdf1['geometry'].apply(check_for_geom)

In [ ]:
gdf1 = gdf1.loc[gdf1['crossed'] == False]

In [ ]:
gv.Polygons(
    gdf1, vdims=[]
).opts( projection=ccrs.Orthographic(27, 48) )

In [ ]:
gv.Polygons(gdf1, vdims=[]).opts(projection=ccrs.Robinson() )